# Week 3 Exercise Questions

## Today's Session Outline:-
1. Data Loading
    * textFile
    * parallelize
    * from Master in a distributed environment (Week 3)
2. MapReduce
    * transformation & actions
        * map & collect
        * filter & count
        * flatMap & take
        * filter & reduce
        * union, intersection
        * join, leftOuterJoin, cartesian, cogroup
        * reduceByKey & collect
3. Config
    * using external files (Week 3)
    * using API
    * Spark Session (discussed with Spark SQL)
4. Partitions
    * fill it with random data
    * find/use the partitions    
5. Spark SQL
    * Basic operations

## Things to remember:- 
* Learn your hardware configuration like RAM, CPU cores, etc.
* All Spark functions will be given along with the questions, you have to fill Spark function with their respective parameters <br /> and write the corresponding Scala or Python Logic
* This is a practice session, so no scores are calculated 
* For quicker programming, we will use the shell environment today
* If your IDE configurations aren't working, approach us after the exercise session
* Ofcourse, Solutions will be provided for these questions after this exercise session
* If you are already familiar with the contents listed above, go ahead in learning Spark SQL

## Configuration:-

https://github.com/shathil/BigDataExercises

From the above url, download the git repository as a zip file. From the dowloaded zip package, extract a folder called resources and place it inside your spark 2.1 folder. 

## Spark Cluster
http://spark.apache.org/docs/1.1.0/cluster-overview.html

### Standalone
http://spark.apache.org/docs/1.1.0/spark-standalone.html
    
### Hadoop YARN
http://spark.apache.org/docs/1.1.0/running-on-yarn.html

### Apache Mesos
http://spark.apache.org/docs/1.1.0/running-on-mesos.html